# Rhyolitic glasses from Mallik et al. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up to find custom python package
import os
import sys
import numpy as np
sys.path.insert(1, ".")
sys.path.insert(1, "..")

In [ ]:
from src import readfiles, wdscan, correct_quant, calczaf, helper_funs
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Nitrogen analyses

### WD scan - visualise & fit

In [ ]:
samplenames = ["A870", "A876", "B989"]
metadata_list = {}
data_list = {}

for s in samplenames:
    print(f"---------------- {s} ----------------")

    folderpath = f"../data/raw/rhyolitic_glasses_StA/{s}_long_scan"
    
    # Read in the data
    for d in ["data001", "data002"]:
        try:             
            comments, data, metadata = readfiles.import_jeol_wdscans(
                subfolder=folderpath,
                scan_filename=f'{d}_mm.csv',
                cnd_filename=f'{d}.cnd',
                comment_line_num=80,
                crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
                sep=',',
                return_metadata=True
            )

            metadata_list[s] = metadata
            data_list[s] = data
        except FileNotFoundError:
                print(f"No file found for {d}")


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 3))
ax.plot(data_list["A870"].L, data_list["A870"].cps_per_nA, label="A870", lw=0.5)
ax.plot(data_list["A876"].L, data_list["A876"].cps_per_nA, label="A876", lw=0.5)
ax.plot(data_list["B989"].L, data_list["B989"].cps_per_nA, label="B989", lw=0.5)
plt.legend()


These are all very similar. I think I could fit them all together and then treat these
three glasses as a 'group' of samples. Maybe just aggregate the 146.6 and 146.4 versions too.

In [ ]:
result, trimmed_data = wdscan.fit_scans_together(
    data = list(data_list.values()),
    fit_regions = [[120, 140], [152, 180]],
    path_out = Path("../data/interim/rhyolitic_glasses/fits")
)

In [ ]:
wdscan.plot_fits_together(
    data=list(data_list.values()),
    trimmed_data=trimmed_data,
    result=result,
    comments=list(data_list.keys()),
    path_out=Path("../data/interim/rhyolitic_glasses/fits")
    )

Those fits look good!

# Quant analysis

In [ ]:
sample_list = ['A870', 'A876', 'B989']
sample_folders = [Path(f'../data/raw/rhyolitic_glasses_StA/quant/{s}/') for s in sample_list]
# List of folders corresponding to the samples
category = 'rhyolitic glasses' # Category of this dataset (e.g. "glasses")
wd_scan = Path(f'../data/interim/rhyolitic_glasses/fits/key_params.txt') # Path to wd scan fit parameters
std_dbase_info_file = Path('data/_dictionaries/standards.csv')

In [ ]:
datalist = readfiles.find_files_and_folders(
                sample_list, sample_folders,
                # apf_file = None,
                apf_file=Path('../data/_dictionaries/apf_values.csv'), #<- Can put None in here
                wd_scan=wd_scan
                )

datalist

In [ ]:
myspot = [None] * len(datalist.folder)

for i in range(len(datalist.folder)):
    peak, bg, standard, info = readfiles.read_and_organise_data(
                                    datalist.loc[i,:].copy(),
                                    bgi=False,
                                    save=False)
    myspot[i] = correct_quant.Spot()
    myspot[i].add_data(info, bg, peak, standard)
    myspot[i].add_wd_scan_params_from_file(wd_scan)
    print('Read dataset:', i + 1, 'of', len(datalist), ':',
          myspot[i].info.comment)
    myspot[i].comprehensify_data()

In [ ]:
correct_quant.process_datasets(
    myspot, 
    datalist, 
    num_mc_sims=100, 
    path_out=Path(f"../data/processed/rhyolitic_glasses/background_corrections")
    )

## Write calczaf files

In [ ]:
samples = sample_list
subfolder = Path(f'../data/processed/rhyolitic_glasses/calczaf_files/')

write_detection_limit_calczaf_files = True
detlim_subfolder = subfolder / Path('detlim')

# note: in the subfolder there must be a file specifying valence.
# this can be copied from the _dictionaries folder.
valence_dict = readfiles.read_valence_file(subfolder, pattern='valence*')
standard_database_dict = pd.read_csv(
    '../data/_dictionaries/standards.csv',
     index_col=0, 
     header=None, 
     squeeze=True).to_dict()

standard_database_dict

In [ ]:
myspot

In [ ]:
# Make a dictionary
# Separate the myspot list by sample
sampledata = [None]*len(samples)
for i, sample in enumerate(samples):
    sampledata[i] = [spot for i, spot in enumerate(myspot) if sample == spot.info['sample']]

sampledata = dict(zip(samples,sampledata))

For these glasses we did not do major element analyses, so we need to use
literature values

In [ ]:
B989_majors = pd.DataFrame.from_dict(dict(
    SiO2 = [67.9, 0.2],
    TiO2 = [0.22, 0.3],
    Al2O3 = [13.58, 0.09],
    FeO = [0.01, 0.01],
    MnO = [0.01, 0.01],
    MgO = [0.25, 0.02],
    CaO = [1.10, 0.04],
    Na2O = [6.0, 0.1],
    K2O = [2.40, 0.05],
    P2O5 = [0.06, 0.04]
), orient="index", columns=["wt%", "stdev"])

A870_majors = pd.DataFrame.from_dict(dict(
    SiO2 = [64.3, 0.3],
    TiO2 = [0.21, 0.2],
    Al2O3 = [13.8, 0.1],
    FeO = [0.03, 0.02],
    MnO = [0.03, 0.02],
    MgO = [0.26, 0.02],
    CaO = [1.07, 0.04],
    Na2O = [5.45, 0.1],
    K2O = [2.38, 0.04],
    P2O5 = [0.05, 0.03]
), orient="index", columns=["wt%", "stdev"])

A876_majors = pd.DataFrame.from_dict(dict(
    SiO2 = [63.4, 0.8],
    TiO2 = [0.23, 0.3],
    Al2O3 = [13.78, 0.01],
    FeO = [0.01, 0.01],
    MnO = [0.01, 0.01],
    MgO = [0.26, 0.02],
    CaO = [1.06, 0.03],
    Na2O = [5.4, 0.4],
    K2O = [2.39, 0.05],
    P2O5 = [0.03, 0.02]
), orient="index", columns=["wt%", "stdev"])

majors_relevant_oxide = dict(zip(["A870", "A876", "B989"], [A870_majors, A876_majors, B989_majors]))

In [ ]:
import pyrolite.geochem

In [ ]:
majors_relevant = {
    k: v[["wt%"]].T[["SiO2", "TiO2", "Al2O3", "FeO", "MnO", "MgO", "CaO", "Na2O", "K2O", "P2O5"]]
    .pyrochem.convert_chemistry(
        to=["Si", "Ti", "Al", "Fe", "Mn", "Mg", "Ca", "Na", "K", "P"]
    )
    .T.squeeze()
    for k, v in majors_relevant_oxide.items()
}

In [ ]:
majors_relevant["B989"]

In [ ]:
# For multiple different methods of processing the data, add a description
run_descriptor = ['_1_base', '_2_bg', '_3_bg_apf']  
# Leave as a list of an empty string if not using: e.g. run_descriptor = ['']

for i in range(len(samples)):

    # Here we pass in these arguments as a dictionary - this is useful in order
    # to reuse the arguments for the detection limit function. But you can
    # alternatively pass in each argument just by defining it in the function
    # as normal (see glasses example).

    args = {
              'elementByDifference' : 'h' # string element symbol
            , 'elementByStoichToStoichOxygen' : None # string element symbol
            , 'stoichOxygenRatio' : 0
            # for hyalophane there is H
            # that can be defined stoichiometrically relative to N:
            , 'elementByStoichToOtherElement' : None
            , 'OtherElement' : None
            , 'stoichElementRatio' : None

            , 'correct_bg' : False
            , 'correct_apf' : False

            # Elements to omit from matrix correction
            # (e.g. if analysed but not actually present in sample)
            , 'remove_elements' : None

            , 'definedElements' : majors_relevant[samples[i]].index # list of element symbols to add
            , 'definedElementWts' : majors_relevant[samples[i]].values # list of known element wt% to add
            }
    
    # Make copies of args with different values
    args2 = args.copy()
    args2["correct_bg"] = True
    args2["correct_apf"] = False

    args3 = args2.copy()
    args3["correct_bg"] = True
    args3["correct_apf"] = True

    args_list = [args, args2, args3]

    for j in range(len(run_descriptor)):
        print("******************************************************")
        print(args_list[j]["correct_bg"], args_list[j]["correct_apf"])
        print("******************************************************")

        calczaf_path_out = subfolder / '{}{}.dat'.format(
                                            samples[i], run_descriptor[j])
        open(calczaf_path_out, 'w').close()  # Erase contents of file

        if write_detection_limit_calczaf_files:
            
            detlim_path_out = detlim_subfolder / '{}{}_detlim.dat'.format(
                                            samples[i], run_descriptor[j])
            open(detlim_path_out, 'w').close()  # Erase contents of file

        for spot in sampledata[samples[i]]:

            calczaf.write_calczaf_input(
                spot, calczaf_path_out, valence_dict, standard_database_dict,
                accV=10, calcMode=2, taAngle=40, Oxide_or_Element=1,
                **args_list[j]) # <- **args unpacks the args dictionary defined earlier
                # so that all those arguments are passed into the function
                # without the need to type them all out.

            if write_detection_limit_calczaf_files:
                if args_list[j]['correct_bg']:

                    detlim_spot = correct_quant.create_detection_limit_spot(spot)

                    calczaf.write_calczaf_input(
                        detlim_spot, detlim_path_out, valence_dict, 
                        standard_database_dict,
                        accV=10, calcMode=2, taAngle=40, Oxide_or_Element=1,
                        **args_list[j])
                    
                else:
                    print('\n\nWarning: Not writing detection limit file.' 
                            'Calculating detection limit does not make sense'
                            ' except on background-corrected data. Raw data files' 
                            ' contain an estimate of detection limit without bg'
                            ' correction.\n')
                    
    

## Process calczaf outputs

In [ ]:
folderpath = Path(f'../data/processed/rhyolitic_glasses/calczaf_files/')

helper_funs.check_calczaf_folder_exists(folderpath)
valence_file = sorted(folderpath.glob('valence*'))[0]

results = calczaf.process_calczaf_outputs(folderpath, valence_file)

# For detection limits

results_detlim = calczaf.process_calczaf_outputs(folderpath / 'detlim/', valence_file, detlim=True)

In [ ]:
summary_tables = {}
typical_kratios = {}

for s in samples:
    summary_tables[s] = correct_quant.write_summary_excel_tables(
        sampledata[s], 
        f"../data/processed/rhyolitic_glasses/kraw_summaries_{s}.xlsx"
        )
    
    typical_kratios[s] = pd.DataFrame({
        "K-ratio": [
            summary_tables[s][0]["original.kraw_pcnt"].mean(),
            summary_tables[s][0]["montecarlo.kraw_pcnt"].mean(),
            summary_tables[s][0]["montecarlo.kraw_apf_pcnt"].mean()
        ],
        "Stdev % (relative)": [
            max(
                summary_tables[s][0]["original.kraw_stdev_pcnt"].mean(),
                summary_tables[s][0]["original.kraw_pcnt"].std()
                ),
            max(
                summary_tables[s][0]["montecarlo.kraw_stdev_pcnt"].mean(),
                summary_tables[s][0]["montecarlo.kraw_pcnt"].std()
                ),
            max(
                summary_tables[s][0]["montecarlo.kraw_stdev_apf_pcnt"].mean(),
                summary_tables[s][0]["montecarlo.kraw_apf_pcnt"].std()
                )
        ]
    }, index = [
        "Original K-ratio (%)", 
        "Bg-corrected K-ratio (%)", 
        "Bg- and APF-corrected K-ratio (%)"
        ]
    )

    typical_kratios[s].insert(
        1, 
        column="Stdev (absolute)", 
        value = typical_kratios[s]["K-ratio"] * typical_kratios[s]["Stdev % (relative)"] / 100
        )
    
    typical_kratios[s]["filename"] = [k for k in results["wtdata"].keys() if s in k]
    typical_kratios[s].reset_index(inplace=True)
    typical_kratios[s].set_index("filename", inplace=True)


In [ ]:
typical_kratios


Plot all spots grouped by their method

In [ ]:
suffix = "3_bg_apf"
stdev_string = "kraw_stdev_apf_pcnt"

In [ ]:
results_apf = {k: v.loc["N", :] for k, v in results["wtdata"].items() if suffix in k}
results_detlim_apf = {k: v.loc["N", :] for k, v in results_detlim["wtdata"].items() if suffix in k}

results_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_apf.items()
}

results_detlim_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_detlim_apf.items()
}

for k, v in results_apf.items():
    v.name = "N wt"

for k, v in results_detlim_apf.items():
    v.name = "N detlim"


In [ ]:
datalist_by_sample= dict(list(datalist.groupby("sample")))

In [ ]:
N_by_method = {}
for s in samples:
    N_by_method[s] = pd.DataFrame(
        {
            "comment": datalist_by_sample[s]["comment"].reset_index(drop=True),
            "N wt": results_apf[f"{s}_3_bg_apf"],
            "N detlim": results_detlim_apf[f"{s}_3_bg_apf_detlim"],
            "N stdev pct": summary_tables[s][0][f"montecarlo.{stdev_string}"],
            "N stdev abs": (
                results_apf[f"{s}_3_bg_apf"]
                * summary_tables[s][0][f"montecarlo.{stdev_string}"]
                / 100
            )
        }
    )

In [ ]:
for s in samples:
    display(N_by_method[s])

# Make a figure showing all quant values against reference

In [ ]:
avgs = pd.concat(
        [N_by_method[s].loc[:, ["N wt", "N stdev abs"]].mean() for s in samples],
    axis=1,
)

avgs.columns = samples
avgs.columns.name = "sample_name"
avgs = avgs.T.reset_index()

avgs = avgs.rename(columns = {"N wt": "measured", "N stdev abs": "measured_stdev"})

In [ ]:
avgs

Note!!! the stdev on multiple measurements is smaller than the stdev on individual ones.
So we'll report the typical stdev on single measurements - this includes the uncertainty on APF propoagated through.

In [ ]:
reference_vals = pd.DataFrame({
    "sample_name": ["A870", "A876", "B989"],
    "reference": [1.0, 0.5, 0.9],
    "reference_stdev": [0.4, 0.3, 0.3],
})

glasses_avgs = pd.merge(reference_vals, avgs, on = "sample_name")

In [ ]:
glasses_avgs

In [ ]:
fig, ax = plt.subplots(figsize=(3,3), )

ax.errorbar(
    x=glasses_avgs["reference"], 
    xerr=glasses_avgs["reference_stdev"],
    y=glasses_avgs["measured"],
    yerr=glasses_avgs["measured_stdev"],
    fmt="ok")

xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
line_min, line_max = (min(xmin, ymin), max(xmax, ymax))
ax.plot([xmin, xmax], [xmin, xmax], "-r", linewidth=0.5)

label_locs = [
    (1.2, 0.9), #A870
    (0.3, 0.8), #A876
    (0.7, 1.3), #B989
]

for i in range(len(glasses_avgs.index)):
    x = glasses_avgs.loc[i, "reference"]
    y = glasses_avgs.loc[i, "measured"]
    ax.annotate(
        glasses_avgs.loc[i, "sample_name"], 
        (x,y), 
        xytext=label_locs[i], ha="center",
        # arrowprops={"arrowstyle": "->"},
        bbox=dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.5))
    
plt.ylabel("N wt% (EPMA)")
plt.xlabel("N wt% (literature)")
plt.xticks(np.arange(0, 2.0, 0.5))
plt.yticks(np.arange(0, 2.0, 0.5))
plt.xlim(line_min, line_max)
plt.ylim(line_min, line_max)
ax.set_aspect("equal")
plt.tight_layout()
plt.savefig("../data/processed/rhyolitic_glasses/epma_vs_reference.png")
plt.show()

In [ ]:
suffix = "1_base"
stdev_string = "original.kraw_stdev_pcnt"
results_apf = {k: v.loc["N", :] for k, v in results["wtdata"].items() if suffix in k}
results_detlim_apf = {k: v.loc["N", :] for k, v in results_detlim["wtdata"].items() if suffix in k}

results_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_apf.items()
}

results_detlim_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_detlim_apf.items()
}

for k, v in results_apf.items():
    v.name = "N wt"

for k, v in results_detlim_apf.items():
    v.name = "N detlim"

datalist_by_sample= dict(list(datalist.groupby("sample")))

N_by_method = {}
for s in samples:
    N_by_method[s] = pd.DataFrame(
        {
            "comment": datalist_by_sample[s]["comment"].reset_index(drop=True),
            "N wt": results_apf[f"{s}_{suffix}"],
            "N stdev pct": summary_tables[s][0][f"{stdev_string}"],
            "N stdev abs": (
                results_apf[f"{s}_{suffix}"]
                * summary_tables[s][0][f"{stdev_string}"]
                / 100
            )
        }
    )

df = pd.concat(N_by_method, axis=0).reset_index()
df_base = df.rename(columns = {"level_0": "sample"}).groupby("sample")[["N wt", "N stdev abs"]].mean()


In [ ]:
suffix = "2_bg"
stdev_string = "montecarlo.kraw_stdev_pcnt"
results_apf = {k: v.loc["N", :] for k, v in results["wtdata"].items() if suffix in k}
results_detlim_apf = {k: v.loc["N", :] for k, v in results_detlim["wtdata"].items() if suffix in k}

results_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_apf.items()
}

results_detlim_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_detlim_apf.items()
}

for k, v in results_apf.items():
    v.name = "N wt"

for k, v in results_detlim_apf.items():
    v.name = "N detlim"

datalist_by_sample= dict(list(datalist.groupby("sample")))

N_by_method = {}
for s in samples:
    N_by_method[s] = pd.DataFrame(
        {
            "comment": datalist_by_sample[s]["comment"].reset_index(drop=True),
            "N wt": results_apf[f"{s}_{suffix}"],
            "N stdev pct": summary_tables[s][0][f"{stdev_string}"],
            "N stdev abs": (
                results_apf[f"{s}_{suffix}"]
                * summary_tables[s][0][f"{stdev_string}"]
                / 100
            )
        }
    )

df = pd.concat(N_by_method, axis=0).reset_index()
df_bg = df.rename(columns = {"level_0": "sample"}).groupby("sample")[["N wt", "N stdev abs"]].mean()


In [ ]:
suffix = "3_bg_apf"
stdev_string = "montecarlo.kraw_stdev_apf_pcnt"
results_apf = {k: v.loc["N", :] for k, v in results["wtdata"].items() if suffix in k}
results_detlim_apf = {k: v.loc["N", :] for k, v in results_detlim["wtdata"].items() if suffix in k}

results_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_apf.items()
}

results_detlim_apf = {
    k: v.loc[
        [c for c in v.index if c not in ["average", "stdev", "minimum", "maximum"]]
    ]
    for k, v in results_detlim_apf.items()
}

for k, v in results_apf.items():
    v.name = "N wt"

for k, v in results_detlim_apf.items():
    v.name = "N detlim"

datalist_by_sample= dict(list(datalist.groupby("sample")))

N_by_method = {}
for s in samples:
    N_by_method[s] = pd.DataFrame(
        {
            "comment": datalist_by_sample[s]["comment"].reset_index(drop=True),
            "N wt": results_apf[f"{s}_{suffix}"],
            "N stdev pct": summary_tables[s][0][f"{stdev_string}"],
            "N stdev abs": (
                results_apf[f"{s}_{suffix}"]
                * summary_tables[s][0][f"{stdev_string}"]
                / 100
            )
        }
    )

df = pd.concat(N_by_method, axis=0).reset_index()
df_bg_apf = df.rename(columns = {"level_0": "sample"}).groupby("sample")[["N wt", "N stdev abs"]].mean()


In [ ]:
final_summary = pd.concat([df_base, df_bg, df_bg_apf], axis=1)
final_summary.columns = ["base", "base stdev", "bg", "bg stdev", "bg apf", "bg apf stdev"]
final_summary.round(2).to_csv("../data/processed/rhyolitic_glasses/N_summary.csv")
final_summary